In [1]:
import torch
from PIL import Image
from torch.nn import functional as F
import os
import time
import cv2
from face_model.gpen_model_classic import FullGenerator

In [2]:
from torchvision import utils, transforms
topil = transforms.ToPILImage()
model_state_reminifaceapp = torch.load('/home/wizard/buckets/bsp-ai-science-scratch/nicg/checkpoints/lifting-gpen/faceapp-remini/classic_gpen/029000.pth')
# model_state_reministylegan = torch.load('/home/wizard/buckets/bsp-ai-science-scratch/nicg/checkpoints/lifting-gpen/stylegan-remini/condition_concat_encoder/028000.pth')
model_state_reministylegan = torch.load('/home/wizard/buckets/bsp-ai-science-scratch/nicg/checkpoints/lifting-gpen/faceapp-remini/classic_gpen/040000.pth')
modelfaceapp = FullGenerator(1024, 512, 8).cuda()
modelfaceapp.load_state_dict(model_state_reminifaceapp['g_ema'])
modelstylegan = FullGenerator(1024, 512, 8).cuda()
modelstylegan.load_state_dict(model_state_reministylegan['g_ema'])

SAVE_TEMP_PATH = '/home/wizard/GPEN-smile/temp'

In [3]:
!pip install pydantic
!pip install torchtyping

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
import os
import sys
THIS_DIR = os.getcwd()
DIFFAE_DIR = '/home/wizard/bendai-lib-python/'
sys.path.insert(0, DIFFAE_DIR)
os.chdir(DIFFAE_DIR)
from production.remini.app.handler import ReminiEnhanceHandler, Request
from production.remini.app.main import setup_model
sys.path.insert(0, THIS_DIR)
os.chdir(THIS_DIR)
def make_model() -> ReminiEnhanceHandler:
    print("\nLoading model... ")
    handler = setup_model()
    print("Model loaded!\n\n")
    return handler
def process_img(model, img_path):
    request = Request(
        
            input_path=img_path,
            output_path='/home/wizard/GPEN-smile/temp/temp_remini',
            enhance_config="enhance-a",
            output_format="image/jpeg",
            pipeline_config="production",
            edit_mode='latent_mapper',
            edit_strength=1.0,
            edit_choice='Attractive',
            edit_enabled=False,
        )
    responsefaceapp = modelfaceapp.inference(request)
    responsestylegan = modelstylegan.inference(request)
    return Image.open(responsefaceapp.enhanced_image_path), Image.open(responsestylegan.enhanced_image_path)


ModuleNotFoundError: No module named 'bsp_logging'

In [ ]:
import matplotlib.pyplot as plt
dir = '/home/wizard/GPEN-smile/examples/remini-small'
imgs = []
remini_model = make_model()
for img_path in [os.path.join(dir, i) for i in os.listdir(dir)][5:]:
    input = cv2.imread(img_path, cv2.IMREAD_COLOR)
    input_orig = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)
    input = torch.from_numpy(input).cuda().permute(2, 0, 1).unsqueeze(0)
    img_t = (input/255.-0.5)/0.5
    img_t = F.interpolate(img_t, (1024, 1024))
    img_t = torch.flip(img_t, [1])
    output, _ = model(img_t, 15000)
    grid = utils.make_grid(
                                    output,
                                    nrow=1,
                                    normalize=True,
                                    range=(-1, 1),
                                )
    topil(grid.cpu()).save(os.path.join(SAVE_TEMP_PATH, img_path.split('/')[-1]))
    remini_output = process_img(remini_model, os.path.join(SAVE_TEMP_PATH, img_path.split('/')[-1]))
    fig, axs = plt.subplots(1, 3, figsize = (30,10))
    axs[0].imshow(input_orig)
    axs[1].imshow(topil(grid.cpu()))
    axs[2].imshow(remini_output)
    plt.show()
    del input, img_t, grid, remini_output

In [ ]:
img_path = '/home/wizard/GPEN-smile/examples/orig.png'
input = cv2.imread(img_path, cv2.IMREAD_COLOR)
input_orig = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)
input = torch.from_numpy(input).cuda().permute(2, 0, 1).unsqueeze(0)
img_t = (input/255.-0.5)/0.5
img_t = F.interpolate(img_t, (1024, 1024))
img_t = torch.flip(img_t, [1])
output, _ = model(img_t, 15000)
grid = utils.make_grid(
                                output,
                                nrow=1,
                                normalize=True,
                                range=(-1, 1),
                            )
topil(grid.cpu()).save(os.path.join(SAVE_TEMP_PATH, img_path.split('/')[-1]))
remini_output = process_img(remini_model, os.path.join(SAVE_TEMP_PATH, img_path.split('/')[-1]))

fig, axs = plt.subplots(1, 3, figsize = (30,10)) 
axs[0].imshow(input_orig)
axs[1].imshow(topil(grid.cpu()))
axs[2].imshow(remini_output)
plt.show()

In [ ]:
img_path = '/home/wizard/diffae/imgs_align/davide_coca.png'
input = cv2.imread(img_path, cv2.IMREAD_COLOR)
input_orig = cv2.cvtColor(input, cv2.COLOR_BGR2RGB)
input = torch.from_numpy(input).cuda().permute(2, 0, 1).unsqueeze(0)
img_t = (input/255.-0.5)/0.5
img_t = F.interpolate(img_t, (1024, 1024))
img_t = torch.flip(img_t, [1])
output, _ = model(img_t, 15000)
grid = utils.make_grid(
                                output,
                                nrow=1,
                                normalize=True,
                                range=(-1, 1),
                            )
topil(grid.cpu()).save(os.path.join(SAVE_TEMP_PATH, img_path.split('/')[-1]))
remini_output = process_img(remini_model, os.path.join(SAVE_TEMP_PATH, img_path.split('/')[-1]))

fig, axs = plt.subplots(1, 3, figsize = (30,10)) 
axs[0].imshow(input_orig)
axs[1].imshow(topil(grid.cpu()))
axs[2].imshow(remini_output)
plt.show()
topil(torch.Tensor(input_orig)[0]).save('orig.jpg')
remini_output.save('edited.jpg')
topil(grid.cpu()).save('editednonremini.jpg')